<a href="https://colab.research.google.com/github/MoltenMuffins/MagicSchoolBus/blob/master/ProductTitleClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
#Boilerplate code
import os, json
from glob import glob

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model,load_model,Sequential

from tensorflow.keras.layers import *
from tensorflow.keras import backend as K

import tensorflow as tf

tf.VERSION

'1.13.1'

In [3]:
#Download the files from the Kaggle NSDC page
!wget -qq https://www.dropbox.com/s/dlpf8lirpf69nvb/ndsc-beginner.zip

#Unzip the zip file
!unzip -qq ndsc-beginner.zip

#Remove zipfile, we won't need it anymore
!rm ndsc-beginner.zip

#Check directory contents
!ls

#The files we want are all in the folder ndsc-beginner

rm: cannot remove '_MACOSX': No such file or directory
__MACOSX  ndsc-beginner  sample_data


In [13]:
#Now let's import train.csv using pandas

import pandas as pd

df = pd.read_csv('./ndsc-beginner/train.csv')
print(df)

            itemid                                              title  \
0           307504               nyx sex bomb pallete natural palette   
1           461203  etude house precious mineral any cushion pearl...   
2          3592295                           milani rose powder blush   
3          4460167                etude house baby sweet sugar powder   
4          5853995       bedak revlon color stay aqua mineral make up   
5          6208490                            dr pure whitening cream   
6          6959546                         chanel powder blush malice   
7          8984461                     snail white cream original 100   
8          9184082                          sunprise all proof spf 50   
9         10345338             eyebrow powder nyx satuan rp 15.000 pc   
10        11016411                        monistat chafing relief gel   
11        11425757                       milani rose powder blush tea   
12        13017527                        the balm 